In [2]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import bisect
import sys

%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator
from matplotlib import cm

pd.set_option('display.float_format', lambda x: '%.10f' % x)

In [3]:
inputData = pd.read_csv('../data/compressHybridBlocks.csv.gz', sep=';', compression='gzip', header=None)
inputData.columns = ['termNo', 'df', 'blocks', 'cdf', 'cdf_bytes', 'cdf_blocks', 'cdf_blocks_bytes', 'ewah_blocks', 'ewah_bytes', 'ewah_block_bytes', 'ones']
inputData

,termNo,df,blocks,cdf,cdf_bytes,cdf_blocks,cdf_blocks_bytes,ewah_blocks,ewah_bytes,ewah_block_bytes,ones
0,0,13059642,203,220362,881448,225322,901288,202,718638,205232,11398025
1,1,12060760,115,144277,577108,147548,590192,114,382840,110352,11120312
2,2,11066901,177,207344,829376,212376,849504,176,504784,179872,9619645
3,3,10789754,207,220144,880576,226400,905600,206,661620,211356,9097964
4,4,10286262,122,153331,613324,157047,628188,121,432766,120516,9290667
5,5,10086692,79,109311,437244,111512,446048,78,356236,78312,9441816
6,6,9606809,207,232065,928260,238098,952392,206,590384,209296,7912738
7,7,9440690,124,164571,658284,168284,673136,123,446446,118080,8428604
8,8,9093904,97,132789,531156,136143,544572,96,268818,96384,8306109
9,9,8921736,202,229042,916168,234959,939836,201,538840,204216,7271160


In [4]:
inputData.describe()

,termNo,df,blocks,cdf,cdf_bytes,cdf_blocks,cdf_blocks_bytes,ewah_blocks,ewah_bytes,ewah_block_bytes,ones
count,1107205.0000000000,1107205.0000000000,1107205.0000000000,1107205.0000000000,1107205.0000000000,1107205.0000000000,1107205.0000000000,1107205.0000000000,1107205.0000000000,1107205.0000000000,1107205.0000000000
mean,553602.0000000000,5400.7301637908,1.2043253056,648.1488396458,2592.5953585831,656.6666678709,2626.6666714836,0.2042169246,393.8260791814,199.3106823036,3170.8685979561
std,319622.6967365845,94050.8872373574,3.0455410148,4961.9156518737,19847.6626074950,5069.9623767691,20279.8495070765,3.0447068383,5970.6324514407,3067.9874656338,73272.6659403269
min,0.0000000000,100.0000000000,1.0000000000,2.0000000000,8.0000000000,2.0000000000,8.0000000000,0.0000000000,2.0000000000,0.0000000000,0.0000000000
25%,276801.0000000000,181.0000000000,1.0000000000,85.0000000000,340.0000000000,85.0000000000,340.0000000000,0.0000000000,2.0000000000,0.0000000000,0.0000000000
50%,553602.0000000000,230.0000000000,1.0000000000,120.0000000000,480.0000000000,120.0000000000,480.0000000000,0.0000000000,18.0000000000,0.0000000000,22.0000000000
75%,830403.0000000000,465.0000000000,1.0000000000,192.0000000000,768.0000000000,192.0000000000,768.0000000000,0.0000000000,48.0000000000,0.0000000000,90.0000000000
max,1107204.0000000000,13059642.0000000000,237.0000000000,277297.0000000000,1109188.0000000000,282284.0000000000,1129136.0000000000,236.0000000000,718638.0000000000,238832.0000000000,11398025.0000000000


In [7]:
inputData[['blocks', 'cdf_bytes', 'cdf_blocks_bytes', 'cdf', 'cdf_blocks']]
#inputData[['ewah_blocks', 'ewah_bytes', 'ewah_block_bytes']]
d = inputData['cdf_blocks_bytes']/inputData['cdf_bytes']
d.describe()

count   1107205.0000000000
mean          1.0003819997
std           0.0025928228
min           1.0000000000
25%           1.0000000000
50%           1.0000000000
75%           1.0000000000
max           1.0594184576
dtype: float64

In [5]:
inputData[['ewah_blocks', 'ewah_bytes', 'ewah_block_bytes']]
#d = inputData['ewah_block_bytes']/inputData['ewah_bytes']
#d.describe()

,ewah_blocks,ewah_bytes,ewah_block_bytes
0,202,718638,205232
1,114,382840,110352
2,176,504784,179872
3,206,661620,211356
4,121,432766,120516
5,78,356236,78312
6,206,590384,209296
7,123,446446,118080
8,96,268818,96384
9,201,538840,204216
